### With statement

Context Manager objects are designed to handle a `with` statement. 

Context managers in Python are constructs that allow you to set up and clean up resources precisely and safely.They can be used for handling databases, locks in concurrent programming or any other resource.

Context manager object should implement `__enter__()` and `__exit__()` functions. at the end of with block the exit function is called. 

In [2]:
import sys

class LookingGlass:
    def __enter__(self):
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return "JABBERWOCKY"
    
    def reverse_write(self, text):
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print("please do not divide by zero!")
            return True

In [4]:
with LookingGlass() as mi:
    print('Hello World!')
print("This is normal!")
print(mi)

!dlroW olleH
This is normal!
JABBERWOCKY


In [2]:
open('test.py', '+r').__enter__()

<_io.TextIOWrapper name='test.py' mode='+r' encoding='UTF-8'>

Using `@contextmanager` reduces the boilerplate of creating a context manager:
instead of writing a whole class with `__enter__/__exit__` methods, you just implement a generator with a single yield that should produce whatever you want the `__enter__` method to return.


In a generator decorated with `@contextmanager`, yield splits the body of the function in two parts: everything before the yield will be executed at the beginning of the with block when the interpreter calls `__enter__`; the code after yield will run when `__exit__` is called at the end of the block.

In [3]:
import contextlib
import sys

@contextlib.contextmanager
def looking_glass():
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    yield "JABBERWOCKY"
    sys.stdout.write = original_write

In [5]:
with looking_glass() as what:
    print('Alice, kitty and snowdrop')
    print(what)

print(what)
print('Back to normal')

pordwons dna yttik ,ecilA
YKCOWREBBAJ
JABBERWOCKY
Back to normal


In [7]:
@contextlib.contextmanager
def foo():
    print('inside enter the context')
    yield 'Atiyeh'
    print("inside the close context")

with foo() as Ati:
    print('inside context manager')
    print(Ati)

print(Ati)

inside enter the context
inside context manager
Atiyeh
inside the close context
Atiyeh


A little-known feature of `@contextmanager` is that the generators decorated with it can also be used as decorators themselves

In [9]:
@looking_glass()
def temp():
     print('The time has come.')

temp()
print('Back to normal!')

.emoc sah emit ehT
Back to normal!


In [11]:
from contextlib import contextmanager
import io
import os


@contextmanager
def inplace(filename, mode='r', buffering=-1, encoding=None, errors=None,
            newline=None, backup_extension=None):
    """Allow for a file to be replaced with new content.

    yields a tuple of (readable, writable) file objects, where writable
    replaces readable.

    If an exception occurs, the old file is restored, removing the
    written data.

    mode should *not* use 'w', 'a' or '+'; only read-only-modes are supported.

    """

    # move existing file to backup, create new file with same permissions
    # borrowed extensively from the fileinput module
    if set(mode).intersection('wa+'):
        raise ValueError('Only read-only file modes can be used')

    backupfilename = filename + (backup_extension or os.extsep + 'bak')
    try:
        os.unlink(backupfilename)
    except os.error:
        pass
    os.rename(filename, backupfilename)
    readable = io.open(backupfilename, mode, buffering=buffering,
                       encoding=encoding, errors=errors, newline=newline)
    try:
        perm = os.fstat(readable.fileno()).st_mode
    except OSError:
        writable = open(filename, 'w' + mode.replace('r', ''),
                        buffering=buffering, encoding=encoding, errors=errors,
                        newline=newline)
    else:
        os_mode = os.O_CREAT | os.O_WRONLY | os.O_TRUNC
        if hasattr(os, 'O_BINARY'):
            os_mode |= os.O_BINARY
        fd = os.open(filename, os_mode, perm)
        writable = io.open(fd, "w" + mode.replace('r', ''), buffering=buffering,
                           encoding=encoding, errors=errors, newline=newline)
        try:
            if hasattr(os, 'chmod'):
                os.chmod(filename, perm)
        except OSError:
            pass
    try:
        yield readable, writable ## The end of the __enter__
    except Exception:
        # move backup back
        try:
            os.unlink(filename)
        except os.error:
            pass
        os.rename(backupfilename, filename)
        raise
    finally:
        readable.close()
        writable.close()
        try:
            os.unlink(backupfilename)
        except os.error:
            pass

In [13]:
import csv

with inplace('test.csv', 'r', newline='') as (infh, outfh):
    reader = csv.reader(infh)
    writer =  csv.writer(outfh)
    
    for row in reader:
        row += ['new', 'columns']
        writer.writerow(row)

#### Question 1:
Implement a context manager in two ways (class-based and function-based) that temporarily changes the working directory to a given path, and returns to the original directory when done.


In [10]:
import os
from contextlib import contextmanager

@contextmanager
def change_directory(path):
    current_path = os.getcwd()
    os.chdir(path)
    print(os.getcwd())
    yield "Changed Directory"
    os.chdir(current_path)
    print(os.getcwd())

Feedbacks on answer:
✅ Minor: You don’t need to print(os.getcwd()) unless for debugging (but it's harmless here).
✅ Problem: If an exception occurs inside yield, the final os.chdir(current_path) might not execute properly if not placed in a try/finally.

``` import os
from contextlib import contextmanager

@contextmanager
def change_directory(path):
    current_path = os.getcwd()
    os.chdir(path)
    print(f"Changed to {os.getcwd()}")
    try:
        yield "Changed Directory"
    finally:
        os.chdir(current_path)
        print(f"Returned to {os.getcwd()}")
```

In [11]:
with change_directory('.venv/') as m:
    print(os.listdir(os.getcwd()))

print(m)

/home/atiyehghm/Desktop/Building-LLM_book/.venv
['lib64', 'bin', 'lib', 'pyvenv.cfg', 'include', 'share']
/home/atiyehghm/Desktop/Building-LLM_book
Changed Directory


In [28]:
class ChangeDirectory:
    def __init__(self, path):
        self.current_path = os.getcwd()
        self.change_path = path

    def __enter__(self):
        os.chdir(self.change_path)
        print(f"Changed to {os.getcwd()}")
        return "Changed Directory!"
    
    ## the exit method should always have these four arguments
    def __exit__(self, exc_type, exc_val, exc_tb):
        os.chdir(self.current_path)
        print("Done with changed directory!")

In [31]:
with ChangeDirectory('/home/atiyehghm/Desktop/Building-LLM_book/temp') as m:
    print(os.listdir(os.getcwd()))

print(os.getcwd())
print(m)

Changed to /home/atiyehghm/Desktop/Building-LLM_book/temp
['temp.py']
Done with changed directory!
/home/atiyehghm/Desktop/Building-LLM_book/.venv
Changed Directory!


### match/case

In [17]:
from collections import ChainMap

m1 = {'a':1, 'b':2, 'c': 3}
m2 = {'a':2, 'z': 3, 'y':6}

cm = ChainMap(m2, m1)
cm['a']

2

In [14]:
x = ['Hello, ', 'maryam']
x = 'Hello'
match x:
    case int(x):
        print('Int')
    case float(x):
        print('Float')
    case ['Hello, ', name]:
        print(name)
    case 'Hello':
        print(x)

Hello


As I've found out there is no pattern matching for a string directly. but we can use regex to use it.

In [ ]:
import re

def check_string(s):
    match s:
        case s if re.match(r"\d{4}-\d{2}-\d{2}", s):
            print("This is a date in YYYY-MM-DD format.")
        case s if re.match(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", s):
            print("This is an email address.")
            
        case _:
            print("No match found.")

In [33]:
check_string('atiyeh1997@gmail.com')
check_string('2025-03-01')

This is an email address.
This is a date in YYYY-MM-DD format.


### `else` Block

the else clause can be used **not only** in `if` statements but also in `for`, `while`, and `try` statements.

**for**


The else block will run only if and when the for loop runs to completion (i.e.,not if the for is aborted with a break).


**while**


The else block will run only if and when the while loop exits because the condition became falsy (i.e., not if the while is aborted with a break).


**try**


The else block will run only if no exception is raised in the try block.


In [ ]:
## the example of the try/ except

try: 
    dangerous_call()
except OSError:
    log('OSError...')
else:
    after_call()

 ##In this exmaple the after_call runs if there are no exceptions in try Block   

**To Know**: python uses the **EAFP** approach that assumes the existence of valid keys or attributes. (Easier to Ask for forgiveness than permission.) On the other hand, languages like C use **LBYL** approach which tests for pre-conditions before-hand.(look beafore you leap.)

#### Question 2
Write a function that takes a list of integers. For each number, use match-case to print:

"Prime" if the number is prime

"Even" if it is even (but not prime)

"Odd" otherwise

In [45]:
def is_prime(a):
    for i in range(2, a//2 + 1):
        if a % i == 0:
            return False
    return True

def odd_even_prime(arr):
    for a in arr:
        match a:
            case a if is_prime(a):
                print("Prime")
            case a if a % 2 == 0:
                print("Even")
            case a if a % 2 == 1:
                 print('Odd')
    else:
        print("All Numbers Are Processed!")

In [46]:
odd_even_prime([2, 4,6, 71, 28, 17 , 33, 89])

Prime
Even
Even
Prime
Even
Prime
Odd
Prime
All Numbers Are Processed!


#### Question3
Write a function that repeatedly asks the user to enter commands until they type "quit". Use match-case to handle:

"hello" → print "Hi there!"

"bye" → print "Goodbye!" and break the loop

any other command → print "Unknown command"

In [ ]:
def screen():
    while True:
        text = input()
        match text:
            case "hello":
                print('Hi there!')
            case "bye":
                print('Goodbye!')
            case 'quit':
                break
            case _:
                print("Unkown Command")
    else:
        print('Exited normally without saying goodbye')
        return
                

In [ ]:
screen()

Unkown Command
Unkown Command
Unkown Command
Unkown Command
Unkown Command
